In [2]:
!pip install pandas scikit-learn lightgbm


In [3]:
from google.colab import drive

import pandas as pd
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import lightgbm as lgb
import re
import string
from sklearn.model_selection import cross_val_predict, StratifiedKFold, GridSearchCV

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd drive/MyDrive/Cs412/project_files

/content/drive/MyDrive/Cs412/project_files


In [6]:
%ls

 bugs-test.csv                       randomforest_predictions_2.csv           submission_0_51.csv
 bugs-train.csv                      random_forest_smote_model.pkl            submission_0_55.csv
 lgbm_p-_sm-.ipynb                   random_forest_smote_model_v1_0.pkl       submission.csv
 ngram_vectorizer.pkl                random_forest_smote_model_v2_tfidf.pkl   tfidf_vectorizer.pkl
'predictions_02 06 2024_02_49.csv'   results/                                 train_prep.csv
 predictions.csv                    'submission_02 06 2024.csv'


In [7]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers'''
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('\n', '', text)
    return "".join(text)

In [8]:

# Load the data
train_data = pd.read_csv('bugs-train.csv')
test_data = pd.read_csv('bugs-test.csv')
# Prepare the data
X = train_data['summary']
test_data['summary'] = test_data['summary']
y = train_data['severity']

In [ ]:
output_file_path = 'train_prep.csv'
train_data.to_csv(output_file_path,index=False)


Output hidden; open in https://colab.research.google.com to view.

In [9]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import LabelEncoder

# Convert text data to TF-IDF features
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(X)
# Encode the severity labels using LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)



In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm
from sklearn.metrics import classification_report, make_scorer, precision_score
# Define the parameter grid
param_grid = {
    'num_leaves': [31, 50],
    'max_depth': [-1, 10, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200]
}
scorer = make_scorer(precision_score, average='macro')

model = lgb.LGBMClassifier(n_jobs=-1, verbose = 3)
# Stratified 5-fold cross-validation with grid search
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring=scorer, n_jobs=-1, verbose=3)

# Fit the grid search
grid_search.fit(X_tfidf, y_encoded)

# Print best parameters and best score
print("Best parameters found: ", grid_search.best_params_)
print("Best cross-validation score: ", grid_search.best_score_)

# Predict using the best estimator
best_model = grid_search.best_estimator_
# Stratified 5-fold cross-validation with progress tracking
predictions = []
for train_index, test_index in tqdm(cv.split(X_tfidf, y_encoded), total=cv.get_n_splits()):
    X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)
    predictions.extend(y_pred)

    print(f"Classification Report for current fold:")
    print(classification_report(y_test, y_pred))

# Overall classification report
predictions = np.array(predictions)
print("Overall Classification Report:")
print(classification_report(y_encoded, predictions))

# Train the final model on the full training set and predict on the test set
best_model.fit(X_tfidf, y_encoded)
test_tfidf = tfidf.transform(test_data['summary'])
test_predictions = best_model.predict(test_tfidf)

# Map the predictions back to their string values
test_data['severity_mapped'] = test_predictions
test_data['severity'] = le.inverse_transform(test_data['severity_mapped'])

# Prepare the output data
test_data.rename(columns={"bug id": "bug_id"}, inplace=True)
output_data = test_data[['bug_id', 'severity']]
output_file_path = 'randomforest_predictions_2.csv'
output_data.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [ ]:

# LightGBM model
model = lgb.LGBMClassifier(n_jobs=-1)

# Stratified 5-fold cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
predictions = cross_val_predict(model, X_tfidf, y_encoded, cv=cv)

# Print classification report for each fold
for i, (train_index, test_index) in enumerate(cv.split(X_tfidf, y_encoded)):
    X_train, X_test = X_tfidf[train_index], X_tfidf[test_index]
    y_train, y_test = y_encoded[train_index], y_encoded[test_index]

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    print(f"Classification Report for Fold {i + 1}:")
    print(classification_report(y_test, y_pred))

# Overall classification report




[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.689394 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 217497
[LightGBM] [Info] Number of data points in the train set: 127998, number of used features: 4703
[LightGBM] [Info] Start training from score -5.430049
[LightGBM] [Info] Start training from score -2.148910
[LightGBM] [Info] Start training from score -3.587888
[LightGBM] [Info] Start training from score -3.274480
[LightGBM] [Info] Start training from score -3.942950
[LightGBM] [Info] Start training from score -0.240038
[LightGBM] [Info] Start training from score -4.889717
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.479712 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 217559
[L

In [ ]:
print("Overall Classification Report:")
print(classification_report(y_encoded, predictions))

Overall Classification Report:
              precision    recall  f1-score   support

           0       0.28      0.04      0.07       701
           1       0.80      0.78      0.79     18658
           2       0.65      0.02      0.05      4426
           3       0.72      0.06      0.12      6053
           4       0.59      0.01      0.03      3102
           5       0.87      0.97      0.92    125854
           6       0.21      0.01      0.03      1204

    accuracy                           0.86    159998
   macro avg       0.59      0.27      0.28    159998
weighted avg       0.84      0.86      0.82    159998



In [ ]:
# Train the final model on the full training set and predict on the test set
model.fit(X_tfidf, y)
test_tfidf = tfidf.transform(test_data['summary'])
test_predictions = model.predict(test_tfidf)

# Save the test predictions with mapping back to original severity strings
test_data['severity_mapped'] = test_predictions
test_data['severity'] = le.inverse_transform(test_data['severity_mapped'])

test_data.rename(columns={"bug id": "bug_id"}, inplace=True)
output_data = test_data[['bug_id', 'severity']]
output_file_path = 'lgb_predictions_2.csv'
output_data.to_csv(output_file_path, index=False)

print(f"Predictions saved to {output_file_path}")